## libraries and function 

In [1]:
from sklearn import datasets
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as skLDA
from scipy import stats
import numpy as np
import pandas as pd
import time 
import math

# RESI

In [2]:
def compute_attribute_weights(A):
  from sklearn.preprocessing import minmax_scale
  y = minmax_scale(A, axis = 0)
  p = y.copy()
  ysum = y.sum(axis = 0)
  for i in range(y.shape[1]):
    if (ysum[i] == 0):
      p[:, i] = 1
    else:
      p[:, i] = y[:, i] / ysum[i]
  from scipy.special import xlogy
  E = - xlogy(p,p).sum(axis=0) / math.log(p.shape[0])
  w = (1 - E)/(len(E) - E.sum())
  if np.isnan(w).any():
    return None
  else:
    return w                                            
def generate_tuple_partition(CT, ICT, m):
  w = compute_attribute_weights(CT)
  r = np.ones(ICT.shape[0])
  for i in range(ICT.shape[0]):
    for j in range(ICT.shape[1]):
      if np.isnan(ICT[i,j]):
        r[i] -= w[j] #If NoneType then insufficient CT set has been used
  ICT = ICT[r.argsort()[::-1],:]
  return np.array_split(ICT, m)
def resi(Xtr_nan, m, n_neighbors):
  CT = [Xtr_nan[~np.isnan(Xtr_nan).any(axis=1)]]
  Tp = []
  T = generate_tuple_partition(CT[0],Xtr_nan[np.isnan(Xtr_nan).any(axis=1)], m)
  from sklearn.impute import KNNImputer
  for i in range(m):
    imputer = KNNImputer(n_neighbors=n_neighbors)
    imputer.fit(CT[-1])
    Tp.append(imputer.transform(T[i]))
    CT.append(np.concatenate((CT[-1],Tp[-1])))
  Tpp = []
  for i in range(m):
    train = CT[0]
    for j in range(1,m):
      if j != i:
        train = np.concatenate((train,T[j]))
    imputer = KNNImputer(n_neighbors=n_neighbors)
    imputer.fit(train)
    Tpp.append(imputer.transform(T[i]))
  imputer = KNNImputer(n_neighbors=n_neighbors)
  imputer.fit(CT[0])
  Tpp.append(imputer.transform(T[m-1]))
  CT = [CT[0]]
  for i in range(m):
    CT.append(np.concatenate((CT[-1],np.mean(np.array([Tp[i], Tpp[i]]), axis=0 ))))
  return CT[-1]

### compute_err function 

In [6]:
def err(mus, S, mus_est, S_est):
  er = [np.linalg.norm(mus_est-mus)/mus.size,
        np.linalg.norm(S_est.flatten().flatten()-S.flatten())/S.size]  
  return np.mean(er)  
 
def generate_nan(Xtrain,ytr, missing_rate):
  minimum_complete_samples = 3
  ct_id = np.random.choice(range(Xtrain.shape[0]), size = minimum_complete_samples, replace = False)
  CT = Xtrain[ct_id]
  ICT = Xtrain[[i for i in range(Xtrain.shape[0]) if i not in ct_id]]
  ICTshape = ICT.shape
  na_id = np.random.randint(0,ICT.size, round(missing_rate*ICT.size))
  ICT = ICT.flatten()
  ICT[na_id] = np.nan
  xxx = np.concatenate((CT,ICT.reshape(ICTshape)))
  ytrain = np.hstack((ytr[ct_id], np.array([ytr[i] for i in range(Xtrain.shape[0]) if i not in ct_id])))
  return xxx, ytrain
 
def compute_err(Xtrain, ytrain, G, missing_rate, minimum_complete_samples = 5, runs = 10):  
  e_rate = []
  running_time = []
  for i in range(runs):
    Xtr_nan, ytrain = generate_nan(Xtrain, ytrain, missing_rate)

    scaler = StandardScaler()
    scaler.fit(Xtr_nan)
    Xtr_nan = scaler.transform(Xtr_nan)
    Xtrain = scaler.transform(Xtrain)
    
    print('rank',np.linalg.matrix_rank(np.nan_to_num(Xtr_nan, copy=True, nan=0.0)))
 
    # estimate parameters from full data
    # each row is a mean of a class
    mus = np.array([np.mean(Xtrain[ytrain==g,:], axis=0) for g in np.arange(G)])
    S = np.array([np.cov(Xtrain[ytrain==g,:],rowvar =False) 
             for g in np.arange(G)])
    print('original mus',mus)
    print('S', S)
 
    #RESI approach
    start = time.time()
    Xtr_resi = resi(Xtr_nan, 3, 3) #Parameters: (Dataset, m, n_neighbors) 
    mus_resi = np.asarray([np.mean(Xtr_resi[ytrain==g,:], axis=0) for g in np.arange(G)])
    S_resi = np.asarray([(sum(ytrain==g))*np.cov(Xtr_resi[ytrain==g,:], rowvar =False)
             for g in np.arange(G)])   
    resi_err = err(mus, S, mus_resi, S_resi)
    resi_time = time.time()-start
    e_rate.append(resi_err)
    running_time.append(resi_time)
  e_rate = np.asarray(e_rate)
  running_time = np.asarray(running_time)
  return np.mean(e_rate), np.std(e_rate), np.mean(running_time)

# Digits

In [4]:
digits = datasets.load_digits()
X,y = digits.data, digits.target.ravel() 
print(X.shape)
rmid = np.where(sum(X!=0)<10)
X = np.delete(X, rmid,axis = 1)
X.shape

(1797, 64)


(1797, 54)

In [7]:
G = 10
e20 = compute_err(X, y, G, 0.2, runs = 10)
e35 = compute_err(X, y, G,0.35, runs = 10)
e50 = compute_err(X, y, G,0.5, runs = 10)
e65 = compute_err(X, y, G,0.65, runs = 10)
e80 = compute_err(X, y, G,0.8, 3, runs = 10)
RESI_res = np.asarray(list((e20, e35, e50, e65, e80)))

rank 54
original mus [[-1.49526006e-02  4.91098739e-02  6.96044905e-03 -1.36341930e-01
  -1.26495379e-01 -1.32705517e-01 -7.25520275e-02 -8.62251978e-02
   3.75688961e-02  6.46176085e-02  1.64492339e-01  2.02391060e-01
  -1.29629592e-01 -4.55818068e-02  6.71332329e-02  1.42164395e-01
  -1.05327890e-02 -6.22372864e-02  2.49287700e-01 -6.50027753e-02
  -8.09728286e-02  1.25004122e-01  9.53569364e-02 -9.16092899e-02
  -2.43866909e-01  4.60240434e-03 -6.45598308e-03 -7.35950743e-03
  -1.21332438e-01 -2.99928736e-01 -3.73895447e-01 -2.02331579e-01
   1.77903332e-02 -8.15976268e-02 -4.10898992e-02 -2.46943353e-01
  -1.60977595e-01 -3.34125889e-02  2.56370048e-02 -9.66472732e-02
   4.01261224e-03  1.65772684e-01  9.04306918e-02  1.49963981e-01
   2.08125996e-01 -1.98060715e-02 -5.40977414e-03 -7.39876587e-02
  -3.45362573e-02  9.73865252e-02  2.03690887e-01  1.53770535e-01
   4.95225551e-02  6.91744893e-02]
 [ 7.10288962e-02 -8.77161812e-05 -4.60529274e-02  1.26531756e-02
   1.81387231e-02  5

rank 54
original mus [[ 1.09346880e-02  1.87761939e-01  1.18288073e-01 -1.67106855e-01
  -1.43864321e-01 -1.49598439e-01  9.01520253e-03  4.49358584e-02
   1.28715127e-01 -5.80390968e-02  1.20324458e-01 -1.35235930e-02
  -2.16801049e-01 -9.29917006e-03  7.99346204e-02  6.13683078e-02
  -7.03578161e-02 -6.42212330e-02  3.63781184e-02 -1.04280516e-01
  -6.87610938e-02  1.34713104e-01  2.01777342e-03 -2.35278942e-01
  -3.19069715e-01 -7.88962236e-02  5.48169687e-02  1.75310996e-01
   1.15291957e-01 -1.27791177e-01 -3.11453565e-01 -2.07976287e-01
   8.72133211e-02  6.49698695e-02  1.99496766e-01 -4.60610172e-02
  -2.30624400e-01  3.12458398e-03  1.00717618e-01 -9.54865173e-02
   3.52180287e-02  3.53186632e-01  1.35948643e-01  1.26138046e-01
   2.59294498e-01  9.87109391e-02 -2.08374293e-04 -1.01001166e-02
   1.54170131e-01  9.52749904e-03  1.34281767e-01  2.38939087e-01
   2.45273122e-01  2.10135663e-01]
 [-1.07484972e-01 -1.88733288e-01  5.85338524e-03  4.78724374e-02
   5.55511186e-02 -1

rank 54
original mus [[-8.60562014e-02  1.14323175e-01  1.02660365e-01 -5.58499597e-02
  -3.05896307e-02 -5.33653388e-02 -5.53015866e-02  1.39029154e-02
   2.20791154e-01 -2.36797209e-02  1.03555148e-01  2.11423287e-01
  -8.17934560e-02 -9.77960103e-02  8.64396152e-02  9.53540643e-02
  -1.86137522e-01 -1.19889697e-01  1.70702260e-01 -7.21737697e-02
  -1.14069375e-01  1.29359108e-01 -4.66940850e-02 -2.23567965e-01
  -2.15027536e-01  4.19696540e-02 -1.78691080e-02  2.71113952e-02
  -5.01286113e-02 -1.43377560e-01 -1.28687055e-01 -1.49911937e-01
  -6.55886197e-02  1.82900418e-02 -4.37841989e-03 -9.38336146e-02
  -4.49731829e-02  6.94078074e-02  4.23183952e-02 -7.18473478e-02
  -2.72601536e-02  1.42498635e-01  2.37316924e-02  5.36039156e-02
   2.10081630e-01  4.24399154e-02  3.99946093e-02 -1.31743496e-01
   8.98935615e-02  1.50402524e-01  7.03735044e-02  3.07664574e-02
   7.27912406e-02  1.82101276e-01]
 [-6.79961312e-02 -1.55509948e-01 -1.19096163e-01 -5.36255603e-02
  -8.04355287e-02 -8

rank 54
original mus [[-7.69238418e-02 -3.33349040e-02 -5.75064929e-03 -1.56507569e-02
   5.74954400e-02  3.86208771e-02  4.47488955e-02 -4.05725762e-02
  -6.98518464e-02 -1.75511111e-01  1.20259674e-01  1.37999183e-01
  -9.03276950e-03  1.32300453e-01 -1.68044361e-01 -1.98254741e-01
  -1.57778547e-02  1.90453715e-01  9.21883481e-02  4.54341435e-02
   9.22719311e-02 -1.66632583e-01 -1.13797049e-01  1.66274955e-02
   1.87165036e-01  6.36249224e-02 -1.83479898e-02 -7.34044679e-02
  -4.53929886e-02 -1.40436890e-01  5.55505621e-02  1.59925908e-01
   2.34401581e-02 -1.57976774e-01 -1.35465963e-01 -1.11909638e-01
   7.06362714e-03  4.33161295e-02  1.48981992e-02 -2.52192530e-03
  -1.75722765e-01 -1.53829442e-01 -1.57636917e-02  1.72071226e-02
   6.78955708e-02 -2.95481482e-02  2.78470496e-03 -5.14821445e-02
   2.85614227e-02 -3.01307592e-02 -3.56876052e-03  1.36050820e-02
   2.78982524e-02  6.43507635e-02]
 [ 1.08445995e-01 -1.28741669e-02  5.41195835e-03 -7.89253371e-02
  -2.61389623e-02 -2

rank 54
original mus [[ 8.99096723e-02  7.77552446e-02  3.12101684e-02 -3.33495329e-02
  -2.10442613e-02  3.33728362e-03  1.26615584e-02  9.02086902e-02
   3.32761910e-02  5.91752249e-02 -6.66995438e-02 -1.21910463e-02
  -4.83259039e-02 -9.70299521e-02  3.14673466e-02  1.06014828e-01
   1.19033158e-02  1.26809547e-02 -8.41535882e-03 -7.96271075e-02
  -9.77283049e-02 -5.87225036e-02 -4.97700601e-02 -1.42645265e-01
  -4.09205847e-03 -1.29305481e-01 -4.94088607e-02 -5.70138805e-02
  -1.00764596e-01 -7.06388333e-02 -2.55542309e-03 -7.63635817e-02
   5.13114376e-02  5.99103092e-02 -1.82045089e-02  8.30981847e-02
   6.50326022e-02 -1.24027242e-01  1.53093126e-02  2.23468796e-02
   4.00296820e-02  1.05754013e-01  2.03436965e-01  1.08472300e-01
  -3.76413375e-02  6.61983174e-02  1.05136231e-01  1.51450377e-02
   1.09559489e-01  7.21932505e-02 -6.57388457e-02  4.09179366e-03
   2.53457481e-02 -3.70672492e-02]
 [-9.42273691e-02  3.38776212e-02 -1.00480870e-02 -3.97190594e-02
   6.03696432e-02  6

...

rank 54
original mus [[-6.33549055e-02 -9.23068644e-02 -8.88530005e-03 -1.61789439e-02
  -6.69302390e-02 -7.68815015e-03 -2.99129899e-02  5.50910415e-03
  -6.27062237e-02  9.69155851e-02  3.48281473e-03 -1.52449529e-01
  -4.86087270e-02 -6.08941608e-02 -6.13696757e-02 -6.10511365e-02
   6.38063727e-02  1.62063022e-02 -1.18020273e-01 -4.49190583e-02
  -1.05071199e-01 -5.62325530e-02  7.86543737e-02  1.17264113e-01
   9.35916799e-02  5.90819440e-03  8.06323887e-03  7.08326271e-02
   6.62482327e-02  5.16156332e-02  7.29324515e-02  1.08929179e-01
   1.01892994e-01  5.11847098e-02  1.31405573e-01  2.08816065e-01
  -2.64229393e-02 -1.22999878e-02  4.37244683e-02  1.79206644e-01
  -9.93664758e-02 -1.39130624e-02  6.41054494e-02  4.67140880e-03
   1.06387711e-02  2.86362321e-02  1.23862303e-01 -1.17582506e-01
  -1.37975919e-01 -2.65587381e-02  4.55051635e-02  8.72050106e-02
  -4.71015180e-02 -5.69445748e-02]
 [-4.50804942e-02  8.89572722e-02  7.20397689e-02 -4.76139369e-02
  -6.90595032e-02 -8

rank 54
original mus [[-2.42472443e-02 -7.57499993e-02  4.52624241e-02 -7.70788301e-02
  -1.85982415e-01 -6.12909118e-02 -8.16010495e-02 -5.81702807e-03
   9.07438211e-05  2.32353737e-02 -1.21995107e-01 -2.11002860e-01
  -3.99758255e-02 -1.04823079e-01 -1.94338447e-02  1.27095739e-02
  -1.59283569e-03 -1.91421673e-02 -1.33271503e-01 -3.03856473e-03
  -3.93610956e-02  7.48889799e-02  4.24471542e-02  7.48051468e-02
  -4.33164322e-02 -1.74286814e-02 -8.42601853e-02  1.67970796e-01
   1.06256967e-01  1.00724300e-01  4.09478187e-02  2.59682889e-02
   5.00140144e-02 -1.53155210e-02  5.67859488e-02  1.24867692e-01
  -1.01791381e-01  9.22542527e-03  5.62294132e-02  8.77903396e-02
  -8.94131367e-02 -3.06142184e-02  6.88742054e-02 -8.65183387e-02
   3.38068387e-02  1.15488002e-01  2.42937585e-01  1.35379262e-02
  -7.33136781e-02  5.21539941e-02  1.01014197e-01  6.90981025e-02
  -3.90833529e-02  1.04920850e-01]
 [-7.06594894e-02  9.59588145e-02  3.47511556e-02  2.08177620e-03
  -1.85885934e-02 -4

rank 54
original mus [[ 4.42686451e-02  5.49417816e-03  6.63774496e-02 -1.15264712e-01
  -8.00776759e-02 -9.80546306e-02 -6.84071951e-02  1.89930461e-04
  -3.04310977e-02  1.16100999e-01 -3.34817527e-02 -7.17378208e-02
  -1.03606532e-01 -1.02716911e-01 -6.29478807e-02  1.82594956e-02
   8.82125284e-02  7.04207194e-02 -5.75548205e-02 -1.28453999e-01
  -4.52333337e-02  2.08269429e-02 -2.79642062e-02  5.81425498e-02
   1.59449040e-02 -3.11111334e-02 -8.15403485e-02 -2.89233021e-02
   7.70067500e-03  6.53590721e-02  9.29114939e-02 -2.64872269e-02
  -1.11739214e-01  5.29760971e-03  8.42552625e-02 -2.91548943e-03
   8.47400274e-02 -6.84803484e-02 -2.35512806e-02  1.53452641e-02
  -4.43211102e-02  1.33076651e-01  3.75706296e-02  1.24223468e-01
  -3.05875509e-02  1.44805894e-01  9.25872320e-02  7.14426292e-02
  -7.73227701e-03 -2.43438921e-02  1.99511321e-02  1.41702250e-01
   1.24161170e-01  1.98519472e-03]
 [ 3.60510674e-02 -3.97792097e-02 -2.80243502e-02 -9.73595544e-03
  -7.78931753e-02  5

rank 54
original mus [[-8.30465685e-02 -6.37263192e-02 -1.04372387e-01  6.53203162e-02
   2.89394524e-02 -9.11289029e-02 -5.57005472e-02 -2.00169493e-03
  -1.29398079e-02  3.98264317e-04 -1.95309811e-03  7.79544228e-02
  -2.42621179e-02 -1.48314554e-02 -5.72263687e-02 -2.93319916e-02
   6.72191913e-03  1.70744501e-02  1.01716535e-01 -8.05677835e-03
   1.02328022e-01  3.87592666e-02  2.32692676e-02 -1.18631479e-02
   3.26548740e-02  6.77429304e-02 -1.28348505e-02 -4.34658818e-02
  -4.12685146e-02 -3.77848018e-02 -2.29619290e-02  1.44156180e-02
   4.94155191e-03  3.97535056e-04  4.66620994e-02 -6.05954888e-02
  -5.52087999e-02 -6.96353033e-03  1.06778378e-01 -8.21020612e-02
   8.78173673e-04 -1.45554049e-02 -9.10419952e-02  2.52207358e-02
   4.40699202e-02 -1.90938172e-02 -1.28442887e-01 -8.00619665e-02
  -4.57092123e-02 -1.05559555e-01  4.61641790e-02  1.64964389e-02
  -1.26049400e-01 -1.06023316e-01]
 [ 7.70319142e-03 -5.09321373e-02 -1.38579082e-01 -4.19366053e-02
  -1.21398567e-01 -3

RESI result: 
 [[1.26244244e-01 5.60537577e-04 5.98726416e-01]
 [1.15282457e-01 6.47694519e-04 7.26782775e-01]
 [1.03863451e-01 1.27058360e-03 9.03602099e-01]
 [9.04968262e-02 1.19559730e-03 1.03118253e+00]
 [7.57532223e-02 9.02641255e-04 1.06863823e+00]]


In [9]:
print('RESI result: \n')
RESI_res.round(3)

RESI result: 



array([[1.260e-01, 1.000e-03, 5.990e-01],
       [1.150e-01, 1.000e-03, 7.270e-01],
       [1.040e-01, 1.000e-03, 9.040e-01],
       [9.000e-02, 1.000e-03, 1.031e+00],
       [7.600e-02, 1.000e-03, 1.069e+00]])